In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"D:\Generative AI\generative ai project\QA_SQL_Tabular_data\data\for_uploads\titanic.csv")
print(df.shape)
print(df.columns.tolist())
display(df.head(3))

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']


,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250


In [3]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
db_path = r"D:\Generative AI\generative ai project\QA_SQL_Tabular_data\data" + "/test_sqldb.db"
db_path = f"sqlite:///{db_path}"

engine = create_engine(db_path)
# df.to_sql("titanic", engine, index=False)
df.to_sql("titanic", engine, index=False)

887

In [4]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM titanic WHERE Age < 2;")

sqlite
['titanic']


"[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]"

In [5]:
df[df["Age"]<2]

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
77,1,2,Master. Alden Gates Caldwell,male,0.83,0,2,29.0000
163,0,3,Master. Eino Viljami Panula,male,1.00,4,1,39.6875
171,1,3,Miss. Eleanor Ileen Johnson,female,1.00,1,1,11.1333
182,1,2,Master. Richard F Becker,male,1.00,2,1,39.0000
303,1,1,Master. Hudson Trevor Allison,male,0.92,1,2,151.5500
379,1,3,Miss. Maria Nakid,female,1.00,0,2,15.7417
384,0,3,Master. Sidney Leonard Goodwin,male,1.00,5,2,46.9000
466,1,3,Miss. Helene Barbara Baclini,female,0.75,2,1,19.2583
641,1,3,Miss. Eugenie Baclini,female,0.75,2,1,19.2583
751,1,2,Master. Viljo Hamalainen,male,0.67,1,1,14.5000


In [6]:
# Create an agent to interact with the Database
from langchain_community.llms import Ollama
client = Ollama(base_url='http://localhost:11434', model='llama3', temperature=0.0)

In [9]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType

agent_executor = create_sql_agent(client, db = db, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)
agent_executor.invoke("what is the average age?")



> Entering new SQL Agent Executor chain...
Let's start by listing all the tables in the database.

Action: sql_db_list_tables
Action Input: empty stringtitanicThought: Now that I know there is a table called "titanic", I should check its schema to see what columns it has. This will help me determine if I can answer the question about average age.

Action: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/Thought: Now that I know the schema of the "titanic" table, I can see that it has an "Age" column which seems relevant to

{'input': 'what is the average age?',
 'output': "The average age is approximately 29.47 years old.\n\nNote: I've limited the query to only return one result (the average age), as per the original question. If you'd like me to provide more information or examples, please let me know!"}

In [10]:
agent_executor.invoke({"input": "what's the average age of survivors"})



> Entering new SQL Agent Executor chain...
Let's start by listing all the tables in the database using `sql_db_list_tables`.

Action: sql_db_list_tables
Action Input: empty stringtitanicThought: Now that I have a list of tables, I should check the schema of the "titanic" table to see what columns it has. This will help me determine if I can query for the average age of survivors.

Action: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/Thought: Now that I have the schema of the "titanic" table, I can see that it has a col

{'input': "what's the average age of survivors",
 'output': '28.408391812865496'}